# Modèle TF-IDF

In [32]:
import pandas as pd
import re
import nltk
import gensim
# from google.colab import drive
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.tokenize import word_tokenize
from gensim.parsing.preprocessing import STOPWORDS
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from itertools import combinations, permutations
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /Users/perrine/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/perrine/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/perrine/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

## Charger le corpus

In [33]:
# drive.mount('/content/drive')

# Choisir le corpus entre "Wikipedia" ou "Stackoverflow"
dataset:str = "Wikipedia"

# Charger le dataset
if dataset == "Stackoverflow":
  dataframe_tfidf = pd.read_json("../data/stackoverflow-data-idf.json", lines=True)
elif  dataset == "Wikipedia":
  dataframe_tfidf = pd.read_json("../data/wikipedia_keywords.json")

# Afficher le schema
print("Schema:\n")
print(dataframe_tfidf.dtypes, "\n")
print("Number of questions,columns=", dataframe_tfidf.shape)

Schema:

keywords    object
sentence    object
dtype: object 

Number of questions,columns= (13014, 2)


In [34]:
# Division du jeu de données en 60% train, 20% dev, 20% test
# temp_df : le reste du corpus à part le train_df
train_df, temp_df = train_test_split(dataframe_tfidf, test_size=0.4, random_state=42)
dev_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# Afficher les dimensions des ensembles pour vérifier
print("Train set:", train_df.shape)
print("Dev set:", dev_df.shape)
print("Test set:", test_df.shape)


Train set: (7808, 2)
Dev set: (2603, 2)
Test set: (2603, 2)


## Prétraitement

In [35]:
# Les fonctions de pré-traitement

stemmer = SnowballStemmer('english')

# Lemmatisation et stemming
def lemmatize_stemming(text:str) -> str:
    """
    Lemmatize and stem the text

    Parameters:
    text : str : text to be lemmatized and stemmed

    Returns:
    str : lemmatized and stemmed text
    """
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='n'))


# Nettoyer le texte
def pre_process(text:str) -> str:
    """
    Convert to lowercase, remove html tags, remove special characters and numbers

    Parameters:
    text : str : text to be cleaned

    Returns:
    text : str : cleaned text
    """
    # Mettre en minuscule
    text = text.lower()
    # Enlever les balises html
    text = re.sub("</?.*?>"," <> ",text)
    # Enlever les caractères spéciaux et numéraux
    text=re.sub("(\\d|\\W)+"," ",text)

    return text

# Fonction pour récupérer la liste des stopwords
def get_stop_words(stop_file_path: str) -> frozenset:
    """
    Load stop words

    Parameters:
    stop_file_path : str : path to the stop words file

    Returns:
    stop_set : set : set of stop words
    """
    with open(stop_file_path, 'r', encoding="utf-8") as f:
        stopwords = f.readlines()
        stop_set = set(m.strip() for m in stopwords)
        return frozenset(stop_set)

stopwords = get_stop_words("../data/stopwords_json.txt")

# Retirer les stopwords depuis le corpus
def stopWords(text: str) -> str:
    """
    Remove stopwords and words with less than 3 characters

    Parameters:
    text : str : text to be cleaned

    Returns:
    result : str : cleaned text
    """
    result = ""
    for token in gensim.utils.simple_preprocess(text):
        if token not in STOPWORDS:
            if token not in stopwords:
                result += lemmatize_stemming(token) + " "
    return result

def no_below(texts:list, min:int) -> list:

    """
    Remove the words which appear in less than {min} documents

    Parameters:
    texts : list of documents(str)
    min : the seuil

    Returns:
    filtered_texts : list of documents(str) : the filtered documents
    """
    word_counts = {}
    for text in texts:
        unique_words = set(text.split())
        for word in unique_words:
            word_counts[word] = word_counts.get(word, 0) + 1

    filtered_texts = []
    for text in texts:
        filtered_text = ' '.join(word for word in text.split() if word_counts[word] > min)
        filtered_texts.append(filtered_text)

    return filtered_texts

# Appliquer la fonction de nettoyage au texte (pré-traitement du corpus)
for df in [train_df, dev_df, test_df]:
  if dataset == "Stackoverflow":
      df['text'] = df['title'] + " " + df['body']
  else:
      df['text'] = df['sentence']
  # Le corpus prêt à être analysé
  df['text'] = df['text'].apply(pre_process).apply(stopWords)
  df['text'] = no_below(df['text'], 20)



# Affichage des résultats du nettoyage
print("Exemple de texte nettoyé :\n")
print(df['text'][:5])

Exemple de texte nettoyé :

10408                                   refer great artist
4099                           histor russian success game
5768     italian defeat germani forc north africa end m...
8974                                        model principl
5350         februari churchil second th british armi base
Name: text, dtype: object


Permet de préparer et de nettoyer du texte pour l'analyse linguistique :

1. **Initialisation du stemmer** :
   - Création d'un outil (`stemmer`) qui simplifie les mots à leur racine. Cela aide à uniformiser différentes formes du même mot.

2. **Fonction de lemmatisation et de stemming (`lemmatize_stemming`)** :
   - Cette fonction transforme un mot pour obtenir sa forme de base (lemmatisation) et ensuite réduit encore cette forme à sa racine (stemming).

3. **Fonction de nettoyage du texte (`pre_process`)** :
   - Transforme tout le texte en minuscules, enlève les balises HTML et les caractères spéciaux. Cela rend le texte plus propre et uniforme pour l'analyse.

4. **Chargement des stopwords** :
   - Les stopwords sont des mots qui sont souvent retirés du texte avant l'analyse car ils sont très fréquents et portent peu d'information utile (comme "the", "and"). La fonction `get_stop_words` charge ces mots depuis un fichier pour les utiliser plus tard.

5. **Fonction de traitement des stopwords (`stopWords`)** :
   - Supprime les mots trop courts ou ceux qui figurent dans la liste des stopwords, applique la lemmatisation et le stemming aux mots restants, puis les assemble en un texte nettoyé.

6. **Application du nettoyage au texte des DataFrames** :
   - Selon le type de données (par exemple, des données de Stackoverflow), le texte est préparé différemment (combinaison de titre et de corps ou utilisation directe des phrases). Chaque texte est ensuite nettoyé et traité pour enlever les mots superflus et simplifier les mots restants.

7. **Affichage des résultats** :
   - Affiche les premiers exemples de texte après nettoyage pour montrer l'efficacité des méthodes de préparation utilisées.

Ce code est utilisé pour améliorer la qualité du texte pour des analyses plus poussées comme la détection de thèmes ou l'analyse de sentiments.

## Création du modèle TF-IDF

In [36]:
# Création du vectorisateur et du transformateur TF-IDF
vectorizer = CountVectorizer(max_df=0.85, max_features=1000)
tfidf_transformer = TfidfTransformer(smooth_idf=True, use_idf=True)

# Apprentissage sur l'ensemble train (transformation TF-IDF sur le train)
train_vectors = vectorizer.fit_transform(train_df['text'])
train_tfidf = tfidf_transformer.fit(train_vectors)

feature_names = vectorizer.get_feature_names_out()

# Transformation des ensembles dev et test
dev_vectors = vectorizer.transform(dev_df['text'])
test_vectors = vectorizer.transform(test_df['text'])
dev_tfidf = tfidf_transformer.transform(dev_vectors)
test_tfidf = tfidf_transformer.transform(test_vectors)

texts = train_df['text'].tolist() # train_df['text'], dev_df['text'] ou test_df['text']

Ce bloc de code configure et utilise des outils pour transformer le texte nettoyé en valeurs numériques, facilitant leur analyse statistique :

1. **Création du vectorisateur et du transformateur TF-IDF** :
   - Un `CountVectorizer` est créé avec des options pour ignorer les mots trop fréquents (plus de 85% des textes) et limiter les mots à considérer à 1000. Cela transforme le texte en une matrice de fréquences de mots.
   - Un `TfidfTransformer` est également préparé pour ajuster ces fréquences en utilisant la méthode TF-IDF, qui pondère les mots en fonction de leur importance dans les documents.

2. **Application sur l'ensemble d'entraînement** :
   - Le vectorisateur est d'abord "entraîné" avec le texte de l'ensemble d'entraînement (`train_df['text']`), convertissant le texte en une matrice de fréquences.
   - Puis, le transformateur TF-IDF est appliqué à cette matrice pour ajuster les fréquences en fonction de leur importance dans l'ensemble du corpus.

3. **Extraction des noms de caractéristiques** :
   - Les noms des mots (features) retenus par le vectorisateur sont extraits, permettant de savoir quels mots ont été considérés dans l'analyse.

4. **Transformation des ensembles de développement et de test** :
   - Les textes des ensembles de développement et de test sont transformés en utilisant le vectorisateur déjà entraîné. Cela assure que les mêmes mots et le même traitement sont utilisés pour tous les ensembles.
   - Les matrices de fréquences obtenues sont ensuite transformées en utilisant le transformateur TF-IDF pour obtenir les pondérations finales.

En résumé, ce code prépare les données textuelles pour des analyses avancées en convertissant les mots en valeurs numériques selon leur fréquence et importance, rendant le texte prêt pour des tâches telles que la classification ou la modélisation de sujets.

In [37]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = nltk.collocations.BigramCollocationFinder.from_words([word for text in texts for word in text.split()])
finder.apply_freq_filter(10)
bigram_score = finder.score_ngrams(bigram_measures.pmi)

bigrams = {}
for bigram, score in bigram_score:
    bigram = '_'.join(bigram)
    bigrams[bigram] = score

# bigrams

In [38]:
trigram_measures = nltk.collocations.TrigramAssocMeasures()
finder = nltk.collocations.TrigramCollocationFinder.from_words([word for text in texts for word in text.split()])
finder.apply_freq_filter(25)
trigram_score = finder.score_ngrams(trigram_measures.pmi)


trigrams = {}
for trigram, score in trigram_score:
    trigram = '_'.join(trigram)
    trigrams[trigram] = score

# trigrams

In [39]:
# Obtenir le texte des colonnes
def get_text_columns(dataset: str, dataframe: str) -> list:
  """
  Function to get the text columns of the dataset

  Parameters:
  dataset: str: the dataset name
  dataframe: DataFrame: the dataset

  Returns:
  list: list of text columns
    sentences: list of sentences
    tags: list of tags
  """
  if dataset == "Stackoverflow":
    titles = dataframe['title'].tolist()
    bodys = dataframe['body'].tolist()
    tags = [ ' '.join([lemmatize_stemming(word) for word in word_tokenize(tag.replace('|', ' ').lower())]) for tag in dataframe['tags'].tolist()]
    return titles, bodys, tags
  elif dataset == "Wikipedia":
    sentences = dataframe['sentence'].tolist()
    tags = [[lemmatize_stemming(keyword.lower()) for keyword in keyword_list] for keyword_list in dataframe['keywords']]
    return sentences, tags

def sort_cooc(cooc_matrix) -> list:
    """
    Function to sort the co-occurrence matrix

    Parameters:
    cooc_matrix: the co-occurrence matrix

    Returns:
    list: sorted co-occurrence matrix
    """
    tuples_cooc = zip(cooc_matrix.col, cooc_matrix.data)
    return sorted(tuples_cooc, key=lambda x: (x[1], x[0]), reverse=True)

def extract_keywords(feature_names: list, sorted_items: list, topn=10) -> dict:
    """
    Extract top N keywords from sorted co-occurrence matrix items.

    Parameters:
    feature_names: list: list of feature names
    sorted_items: list: sorted items
    topn: int: number of keywords to extract

    Returns:
    dict: dictionary of keywords
    """
    sorted_items = sorted_items[:topn]
    keywords = {feature_names[idx]: round(score, 3) for idx, score in sorted_items}
    return keywords

def extract_keywords_from_doc(tfidf_vector, doc_index, vectorizer, transformer, topn=10) -> dict:
    """
    Function to extract keywords from a document

    Parameters:
    tfidf_vector: the tfidf vector
    doc_index: the document index
    vectorizer: the vectorizer
    transformer: the transformer
    topn: the number of keywords to extract

    Returns:
    dict: dictionary of keywords
    """
    row_tfidf_vector = transformer.transform(tfidf_vector[doc_index])
    sorted_items = sort_cooc(row_tfidf_vector.tocoo())
    keywords = extract_keywords(vectorizer.get_feature_names_out(), sorted_items, topn)
    return keywords



1. **Extraction des textes et des tags** :
   - La fonction `get_text_columns` récupère différents types de contenus textuels (titres, corps de texte, tags) selon le type de dataset (par exemple, Stackoverflow ou Wikipedia). Les tags sont nettoyés et simplifiés en utilisant des fonctions de lemmatisation et de stemming pour uniformiser les termes.

2. **Tri d'une matrice de cooccurrence** :
   - La fonction `sort_cooc` trie les éléments d'une matrice de cooccurrence, qui sont des paires index-valeur indiquant la fréquence ou l'importance d'un mot. Le tri se fait de manière à placer les éléments les plus significatifs en premier, basé sur leur score.

3. **Extraction des mots-clés** :
   - La fonction `extract_keywords` sélectionne les principaux mots-clés à partir des éléments triés de la matrice de cooccurrence, limitant le nombre de mots-clés à un nombre spécifié (par exemple, les 10 mots les plus pertinents).

4. **Extraction de mots-clés d'un document spécifique** :
   - `extract_keywords_from_doc` applique les méthodes précédentes à un document spécifique du DataFrame. Elle transforme d'abord le texte en valeurs TF-IDF, trie les résultats, puis extrait et renvoie les mots-clés les plus pertinents pour ce document.

Ces fonctions, ensemble, facilitent l'analyse textuelle en permettant d'extraire et de quantifier l'importance des mots dans des textes.

In [40]:
def print_keywords(idx: int, keywords: dict, columns: list, dataset: str) -> None:
    """
    Function to print the keywords extracted from a document

    Parameters:
    idx: int: the index of the document
    keywords: dict: the keywords extracted
    columns: list: the columns of the dataset
    dataset: str: the dataset name

    Returns:
    None
    """
    if dataset == "Stackoverflow" :
        titles, bodys, tags = columns
        print("=====Title:=====")
        print(titles[idx])
        print("\n=====Text:=====")
        print(bodys[idx])
        print("\n=====Tags:=====")
        print(tags[idx])
        print("\n=====Keywords:=====")
        for k in keywords:
            print(k, keywords[k])
    if dataset == "Wikipedia" :
        sentences, tags = columns
        print("=====Sentences:=====")
        print(sentences[idx])
        print("\n=====Tags:=====")
        print(tags[idx])
        print("\n=====Keywords:=====")
        for k in keywords:
            print(k, keywords[k])

index = 2
data_df = train_df # train_df, dev_df ou test_df
columns = get_text_columns(dataset, data_df)
keywords_and_scores = extract_keywords_from_doc(train_vectors, index, vectorizer, tfidf_transformer, 10)
print_keywords(index, keywords_and_scores, columns, dataset)

=====Sentences:=====
In May, Churchill was still generally unpopular with many Conservatives and probably most of the Labour Party.

=====Tags:=====
['labour parti']

=====Keywords:=====
labour 0.534
conserv 0.496
parti 0.417
churchil 0.41
general 0.357


1. **Fonction `print_keywords`** :
   - Cette fonction affiche les informations pertinentes d'un document spécifique, comme son titre, son texte, ses tags associés, et les mots-clés extraits. Elle adapte son affichage selon que les données proviennent de Stackoverflow ou de Wikipedia, soulignant la flexibilité de la fonction à différents formats de données.
   - Pour chaque mot-clé extrait, la fonction affiche également son score TF-IDF, ce qui indique l'importance du mot dans le document par rapport au corpus total.

2. **Utilisation des fonctions** :
   - Un index spécifique est choisi pour identifier le document à analyser. Ce choix d'index permet de cibler précisément quel document doit être traité.
   - La fonction `get_text_columns` extrait les données textuelles nécessaires (titres, corps, tags) du DataFrame et les prépare pour l'analyse.
   - `extract_keywords_from_doc` applique le traitement TF-IDF au document spécifié, trie les mots en fonction de leur importance, et extrait les mots-clés les plus pertinents.
   - `print_keywords` est ensuite appelée pour afficher les résultats, permettant de visualiser les mots-clés les plus significatifs pour le document sélectionné, ainsi que le contenu associé pour contexte.

3. **Processus détaillé** :
   - Après avoir sélectionné le document via un index, le code récupère les données nécessaires à partir du DataFrame en utilisant une fonction dédiée.
   - Il transforme ensuite ces données en vecteurs TF-IDF pour évaluer l'importance de chaque mot.
   - Les mots-clés sont identifiés en sélectionnant les termes avec les plus hauts scores TF-IDF, et ces mots-clés sont affichés avec leurs scores respectifs pour indiquer clairement leur pertinence.

Ces étapes permettent non seulement d'extraire des informations clés des documents textuels, mais aussi de les présenter de manière claire et organisée, facilitant l'analyse du contenu et la compréhension des sujets principaux des documents.

In [41]:
# Verify if the combination of keywords and permutation is present in bigrams and trigrams dictionary. If present, add the bigram/trigam to the keyword
def verification_bigram_trigram(keywords: dict, bigrams: dict, trigrams: dict) -> dict:
    """
    Verify if the combination of keywords and permutation is present in bigrams and trigrams dictionary. If present, add the bigram/trigam to the keyword

    Parameters:
    keyword : dict : dictionary containing keywords and their scores
    bigrams : dict : dictionary containing bigrams and their scores
    trigrams : dict : dictionary containing trigrams and their scores

    Returns:
    dict : updated dictionary containing keywords and their scores
    """
    updated_keywords = []
    for keyword_dict in keywords:  # Utiliser un nom différent pour l'itérateur
        updated_keyword_dict = keyword_dict.copy()  # Créer une copie du dictionnaire original
        keyword_keys = keyword_dict.keys()  # Utiliser un nom différent pour éviter la confusion
        bigram_set = set(bigrams.keys())  # Obtenir les clés du dictionnaire de bigrammes
        trigram_set = set(trigrams.keys())  # Obtenir les clés du dictionnaire de trigrammes

        combined_keywords = set()
        for k in range(1, min(4, len(keyword_keys) + 1)):  # Utiliser min pour ne pas dépasser 3 tokens
            for subset in combinations(keyword_keys, k):
                for perm in permutations(subset):
                    combined_keyword = '_'.join(perm)
                    if len(combined_keyword.split()) <= 3:  # Vérifier le nombre de tokens
                        combined_keywords.add(combined_keyword)

        for combined_keyword in combined_keywords:
            if combined_keyword in bigram_set or combined_keyword in trigram_set:
                updated_keyword_dict[combined_keyword] = 0.001  # Ajouter une petite valeur pour indiquer qu'il s'agit d'un bigramme/trigramme

        updated_keywords.append(updated_keyword_dict)  # Ajouter le dictionnaire mis à jour à la liste

    return updated_keywords

In [42]:
def calculate_precision_recall_fscore(true_tags: list, predicted_keywords: list) -> tuple:
    """
    Function to calculate precision, recall and f-score

    Parameters:
    true_tags: list: the true tags
    predicted_keywords: list: the predicted keywords

    Returns:
    tuple: precision, recall, f-score
    """
    true_set = set(true_tags)  # Conversion des tags en ensemble
    predicted_set = set(predicted_keywords)  # Les mots-clés sont déjà un ensemble
    tp = len(true_set & predicted_set)
    fp = len(predicted_set - true_set)
    fn = len(true_set - predicted_set)

    precision = tp / (tp + fp) if tp + fp > 0 else 0
    recall = tp / (tp + fn) if tp + fn > 0 else 0
    f_score = 2 * precision * recall / (precision + recall) if precision + recall > 0 else 0

    return precision, recall, f_score

La fonction `calculate_precision_recall_fscore` est conçue pour calculer trois métriques importantes dans l'évaluation des systèmes de récupération d'information ou de classification : la précision, le rappel et la F-mesure (F-score). Voici un détail de chaque étape et de chaque calcul effectué dans la fonction :

1. Conversion des Tags en Ensembles :
* `true_tags` : Une chaîne de caractères contenant les tags corrects, séparés par des espaces. Ces tags sont convertis en un ensemble (set) pour permettre des opérations ensemblistes.
* `predicted_keywords` : Un ensemble de mots-clés prédits. L'utilisation d'un ensemble permet également de réaliser facilement des opérations d'intersection et de différence.

2. Calcul des True Positives (tp), False Positives (fp) et False Negatives (fn) :
* True Positives (tp) : Le nombre d'éléments qui sont à la fois dans les `true_tags` et dans les `predicted_keywords`. Cela représente le nombre de prédictions correctes.
* False Positives (fp) : Le nombre d'éléments qui sont dans `predicted_keywords` mais pas dans `true_tags`. Ces éléments ont été incorrectement prédits comme étant des tags.
* False Negatives (fn) : Le nombre d'éléments qui sont dans `true_tags` mais pas dans `predicted_keywords`. Ces éléments sont des tags corrects qui n'ont pas été prédits.

3. Calcul de la Précision :
* Précision : Cette métrique évalue la qualité des prédictions. Elle est définie comme le ratio des vrais positifs par rapport à la somme des vrais positifs et des faux positifs. Une précision élevée signifie que la majorité des tags prédits sont corrects.
* Formule :
Précision= tp/(tp+fp)

4. Calcul du Rappel :
* Rappel : Cette métrique mesure l'exhaustivité des prédictions. Elle est définie comme le ratio des vrais positifs par rapport à la somme des vrais positifs et des faux négatifs. Un rappel élevé signifie que la majorité des tags corrects ont été capturés par les prédictions.
* Formule :
Rappel = tp/(tp+fn)

5. Calcul de la F-mesure (F-score) :
* F-score : Cette métrique combine la précision et le rappel dans une seule mesure, utilisant leur moyenne harmonique. Elle est particulièrement utile quand il est important d'équilibrer la précision et le rappel, sans favoriser l'une ou l'autre.
* Formule :
F1 = 2x((PrécisionxRappel)/(Précision+Rappel))

6. Retour des Résultats :
La fonction retourne les trois métriques calculées : précision, rappel et F-score.

Cette fonction est cruciale pour évaluer les systèmes de classification ou de recommandation où la correspondance exacte entre les éléments prédits et les vérités terrain est essentielle pour mesurer la performance.


In [43]:
# Extraire les mots-clés de tous les documents
tf_idf_vector = tfidf_transformer.transform(vectorizer.transform(texts))

def process_documents(dataframe, tfidf_vector, vectorizer, transformer) -> list:
    """
    Function to process all documents in the dataset

    Parameters:
    dataframe: DataFrame: the dataset
    tfidf_vector: the TF-IDF vector
    vectorizer: the vectorizer
    transformer: the transformer

    Returns:
    list: list of all keywords with their scores
    """
    all_keywords = []

    for idx in range(dataframe.shape[0]):
        keywords = extract_keywords_from_doc(tfidf_vector, idx, vectorizer, transformer)
        all_keywords.append(keywords)

    return all_keywords

In [44]:
def matches_keywords(keyword_scores: list, tags: list) -> list:
    """
    Check if the keywords match the tags.

    Parameters:
    keyword_scores : list : list of dictionaries containing keywords and their scores
    tags : list : list of tags

    Returns:
    list : list of numbers indicating the count of keyword matches per tag list
    """
    pourcentages:list=[]
    matches = []
    for tag_list, keyword_dict in zip(tags, keyword_scores):
        keywords = list(keyword_dict.keys())
        # print('keywords: ', keywords, '\ntag_list: ', tag_list)
        tag_ngram = []
        # S'il y a des tags de bigrams ou trigrams séparés par espace, on le remplace par '_'
        for tag in tag_list:
            tag = tag.strip().replace(' ', '_')
            # print(tag)
            tag_ngram.append(tag)

        match = 0
        bingo_pourcentage:str = ""
        for keyword in keywords:
            if keyword in tag_ngram:
                match += 1
        bingo_pourcentage = f"{round(match / len(tag_ngram), 3) * 100}%"
        pourcentages.append(bingo_pourcentage)
        matches.append(match)
    return matches, pourcentages

if dataset == "Stackoverflow":
  titles, bodys, tags = get_text_columns(dataset, data_df)
  tags = [tag.split(' ') for tag in tags]
  train_keywords = process_documents(data_df, train_vectors, vectorizer, tfidf_transformer)
  dataframe = pd.DataFrame(zip(titles, bodys, tags), columns=['title', 'body', 'tags'])
  train_keywords = verification_bigram_trigram(train_keywords, bigrams, trigrams)
  # Ajout des résultats au DataFrame
  dataframe['keywords'] = train_keywords
  evaluation_scores = [calculate_precision_recall_fscore(tags[i], train_keywords[i]) for i in range(len(tags))]
  dataframe['precision'], dataframe['recall'], dataframe['f_score'] = zip(*evaluation_scores)
  dataframe['match'], dataframe['match percent'] = matches_keywords(train_keywords, tags)
  # print(dataframe[['title', 'body', 'tags', 'keywords', 'match', 'precision', 'recall', 'f_score']].head())
  dataframe.to_csv("../results/csv/Stackoverflow-data-idf-without-stopwords-match-train_df.csv", index=False)

elif dataset == "Wikipedia":
  sentences, tags = get_text_columns(dataset, data_df)
  train_keywords = process_documents(data_df, train_vectors, vectorizer, tfidf_transformer)
  dataframe = pd.DataFrame(zip(sentences, tags), columns=['sentences', 'tags'])
  train_keywords = verification_bigram_trigram(train_keywords, bigrams, trigrams)
  dataframe['keywords'] = train_keywords
  dataframe['match'], dataframe['match percent'] = matches_keywords(train_keywords, tags)
  evaluation_scores = [calculate_precision_recall_fscore(tags[i], train_keywords[i]) for i in range(len(tags))]
  # print("Evaluation scores: ", evaluation_scores)
  dataframe['precision'], dataframe['recall'], dataframe['f_score'] = zip(*evaluation_scores)
  # print(dataframe[['sentences', 'tags', 'keywords', 'match', 'precision', 'recall', 'f_score']].head())
  dataframe.to_csv("../results/csv/Wikipedia-data-idf-without-stopwords-match-train_df.csv", index=False)


1. **Transformation des documents en vecteurs TF-IDF** :
   - Les documents sont d'abord transformés en vecteurs de mots par le `vectorizer`, puis ces vecteurs sont convertis en représentations TF-IDF grâce au `tfidf_transformer`. Cela permet de préparer les documents pour une analyse plus détaillée basée sur l'importance des mots dans le corpus.

2. **Extraction des mots-clés et évaluation des performances** :
   - Pour chaque document, les mots-clés sont extraits et évalués en utilisant les fonctions `extract_keywords_from_doc` et `calculate_precision_recall_fscore`. Les mots-clés sont choisis en fonction de leur pertinence calculée par TF-IDF, et les performances sont mesurées par la précision, le rappel, et le score F, qui comparent les mots-clés prédits aux tags réels du document.

3. **Calcul du nombre de correspondances entre les tags et les mots-clés** :
   - La fonction `matches_keywords` calcule combien de mots-clés extraits correspondent effectivement aux tags des documents, donnant ainsi une mesure de l'exactitude des mots-clés extraits.

4. **Préparation et sauvegarde des résultats** :
   - Les résultats sont organisés dans un DataFrame de pandas, qui inclut les titres, les textes, les tags originaux, les mots-clés extraits, et les scores de performance pour chaque document.
   - Ces données sont ensuite sauvegardées dans un fichier CSV pour une utilisation ultérieure ou une analyse plus approfondie.

5. **Affichage des résultats** :
   - Les premières lignes du DataFrame sont affichées pour donner un aperçu des mots-clés et des scores associés, permettant de vérifier rapidement l'efficacité de l'extraction et de l'évaluation des mots-clés.


## Test

In [45]:
# Préparation des données de test
test_texts = test_df['text'].tolist()
test_vectors = vectorizer.transform(test_df['text'])
test_tfidf = tfidf_transformer.transform(test_vectors)

# Extraction des mots-clés pour les documents de test
test_keywords = process_documents(test_df, test_vectors, vectorizer, tfidf_transformer)
test_keywords = verification_bigram_trigram(test_keywords, bigrams, trigrams)

# Extraction des colonnes de texte et des tags pour les données de test
if dataset == "Stackoverflow":
    test_titles, test_bodys, test_tags = get_text_columns(dataset, test_df)
    test_tags = [tag.split(' ') for tag in test_tags]
    test_evaluation_scores = [calculate_precision_recall_fscore(test_tags[i], test_keywords[i]) for i in range(len(test_tags))]
    test_match_keywords, test_match_percent = matches_keywords(test_keywords, test_tags)
    test_dataframe = pd.DataFrame(zip(test_titles, test_bodys, test_tags, test_keywords, test_match_keywords, test_match_percent), columns=['title', 'body', 'tags', 'keywords', 'match', 'match percent'])
    test_dataframe['precision'], test_dataframe['recall'], test_dataframe['f_score'] = zip(*test_evaluation_scores)
    test_dataframe.to_csv("../results/csv/Stackoverflow-data-idf-without-stopwords-match-test_df.csv", index=False)
    print(test_dataframe[['title', 'tags', 'keywords', 'match', 'match percent', 'precision', 'recall', 'f_score']].head())

elif dataset == "Wikipedia":
    test_sentences, test_tags = get_text_columns(dataset, test_df)
    test_evaluation_scores = [calculate_precision_recall_fscore(test_tags[i], test_keywords[i]) for i in range(len(test_tags))]
    test_match_keywords, test_match_percent = matches_keywords(test_keywords, test_tags)
    test_dataframe = pd.DataFrame(zip(test_sentences, test_tags, test_keywords, test_match_keywords, test_match_percent), columns=['sentences', 'tags', 'keywords', 'match', 'match percent'])
    test_dataframe['precision'], test_dataframe['recall'], test_dataframe['f_score'] = zip(*test_evaluation_scores)
    test_dataframe.to_csv("../results/csv/Wikipedia-data-idf-without-stopwords-match-test_df.csv", index=False)
    print(test_dataframe[['sentences', 'tags', 'keywords', 'match', 'match percent', 'precision', 'recall', 'f_score']].head())

                                           sentences  \
0  Laurence had interviewed Sweeney and his crew,...   
1  Historically, Russian athletes have been one o...   
2  Italian defeats prompted Germany to deploy an ...   
3  Genotype evolution can be modeled with the Har...   
4  In February 1895, Churchill was commissioned a...   

                                                tags  \
0                                 [the great artist]   
1             [russian athlet, olympic gam, russian]   
2  [deploy an expeditionary forc, rommel, afrika ...   
3                          [hardy-weinberg principl]   
4  [4th queen's own hussar, british armi, aldershot]   

                                            keywords  match  \
0  {'artist': 0.651, 'refer': 0.569, 'great': 0.503}      0   
1  {'game': 0.553, 'histor': 0.542, 'success': 0....      1   
2  {'forc': 0.549, 'africa': 0.261, 'axi': 0.249,...      2   
3                {'principl': 0.782, 'model': 0.623}      0   
4  {'februa